In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tabulate import tabulate
from patsy import dmatrices
import statsmodels.api as sm
from datetime import timedelta

In [ ]:
#ID de bar a predecir
ID = 111401
#dias extra de predicción
dias = 200


In [ ]:
master = pd.read_parquet("./processed_data/3_categorical_master.parquet")

data = master[['fecha_pedido','litros_pedidos_cliente', 'id_cliente']]


data = data.rename(columns={'fecha_pedido':'ds'})
data = data.rename(columns={'litros_pedidos_cliente':'y'})
data = data.rename(columns={'id_cliente':'unique_id'})
data['ds'] = pd.to_datetime(data['ds'])
data['y'] = 1

start_date = data.ds.min()
day_limit = data.ds.max()
end_date = data.ds.max() + timedelta(days=dias)

# Create a MultiIndex with all combinations of product and date
product_range = pd.MultiIndex.from_product([data['unique_id'].unique(), pd.date_range(start=start_date, end=end_date, freq='D')], names=['unique_id', 'ds'])


# Merge with the original DataFrame to fill in missing values
data = pd.merge(data, pd.DataFrame(index=product_range).reset_index(), on=['unique_id', 'ds'], how='right').fillna(0)

#data=data.set_index('ds').reindex(pd.date_range(start=data.ds.min(), end=data.ds.max(),freq='D'),fill_value=0)
#data = data.reset_index()
#data = data.rename(columns={'index':'ds'})
#data


Indiquem el bar id per el que volem predir

In [ ]:
data = data[data['unique_id'] == ID]
data = data.reset_index()
last_day = data[data['ds'] == day_limit]
last_day = last_day.index[0]

Creem les variables externes que afecten a les prediccions (Dia de la setmana, clima...)

In [ ]:
ds = data.ds
data['MONTH'] = ds.dt.month
data['DAY_OF_WEEK'] = ds.dt.dayofweek
data['DAY'] = ds.dt.day

data

Separem les dades en entrenament vs validació

In [ ]:
mask = np.random.rand(len(data)) < 0.8
df_train = data[mask]
df_test = data[~mask]
df_future = data
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

Entrenem un regressor de Poisson

In [ ]:
expr = """y ~ DAY + DAY_OF_WEEK + MONTH"""
y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')


In [ ]:
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

Resultats


In [ ]:
poisson_predictions = poisson_training_results.get_prediction(X_test)
#summary_frame() returns a pandas DataFrame
predictions_summary_frame = poisson_predictions.summary_frame()
print(predictions_summary_frame)

HAY QUE CAMBIAR EL THRESHOLD PARA LO QUE CONSIDERAMOS COMO 1

In [ ]:
predicted_counts=predictions_summary_frame['mean']
actual_counts = y_test['y']
fig = plt.figure()
fig.suptitle('Predicted versus actual data')
predicted, = plt.plot(X_test.index, predicted_counts, 'go-', label='Predicted counts')
actual, = plt.plot(X_test.index, actual_counts, 'ro-', label='Actual counts')
point2, = plt.plot(np.array([i for i in range(0,700)]), np.array([0.19 for i in range(0,700)]))
plt.legend(handles=[predicted, actual])
plt.show()

Aqui está la lista de las siguientes predicciones, no se por que pero parece que los 0's los ignora 


In [ ]:
future = predicted_counts[predicted_counts.index > last_day]
print(future)

In [ ]:
mean = future.sum()/len(future)
future = future[future.values >= mean]
print(future)